In [1]:
import essentia.standard as ess
import numpy as np
from scipy.signal import convolve
import glob, os, sys

In [2]:
import sox
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, AddImpulseResponse

Defining paths

In [6]:
ir_paths = {'MIT':'../Databases/IR/MIT/',
          'AllurePack':'../Databases/IR/AllurePack/',
          'MeldaProduction_Microphones':'../Databases/IR/MeldaProduction_Microphones/',
          'MeldaProduction_Rooms':'../Databases/IR/MeldaProduction_Rooms/',
          'MeldaProduction_Box':'../Databases/IR/MeldaProduction_Box/'}
audio_path = '../Databases/Raw_database/'

## Load IR files

In [6]:
database_name = 'MeldaProduction_Box'
ir_path = ir_paths[database_name]

In [60]:
if ir_path == ir_paths['MIT']: #We only use a subset of IR from MIT IR database
    filtered_ir =['classroom','diningroom','livingroom','bedroom','office','kitchen']
    not_wanted = ['conferenceroom','1txts','1txs','meetingroom','bathroom','hallway','largebrickwalledopenplanoffic','lobby','foyer']
    ir_files = []
    for f in glob.glob(ir_path + '*.wav'):
        f_norm = f.lower()
        if any(room in f_norm for room in filtered_ir):
            if any(room in f_norm for room in not_wanted ):
                pass
            else:
                ir_files.append(f)
else:
    types = ('*.wav', '*.flac') 
    ir_files = []
    for files in types:
        ir_files.extend(glob.glob(ir_path + files))

In [61]:
print('You loaded {} IR files from "{}" database. '.format(len(ir_files),database_name))

You loaded 18 IR files from "MeldaProduction_Box" database. 


## Load Raw Audios
 We will only apply data augmentation on _DI recordings!

In [7]:
raw_audios = glob.glob(audio_path + '*_DI.wav') #+ glob.glob(audio_path + '*_Reference.wav')
sample_rate = 44100

In [9]:
if database_name == 'MIT':#for the naming of the output files
    ir_names = [r.split('_')[0].replace(ir_path,'') for r in ir_files]
else:
    ir_names = []
    for r in ir_files:
        r_norm = r.split('/')[-1].replace('.wav','')
        r_norm = r_norm.replace('.flac','')
        r_norm = r_norm.replace(' ','_')
        ir_names.append(r_norm)

In [64]:
ir_files

['../Databases/IR/MeldaProduction_Box/Mobile Phone 01.wav',
 '../Databases/IR/MeldaProduction_Box/Guitar Cabinet 03.wav',
 '../Databases/IR/MeldaProduction_Box/Guitar Cabinet 02.wav',
 '../Databases/IR/MeldaProduction_Box/Voice Recorder 01.wav',
 '../Databases/IR/MeldaProduction_Box/Guitar Cabinet 01.wav',
 '../Databases/IR/MeldaProduction_Box/Laptop Speaker.wav',
 '../Databases/IR/MeldaProduction_Box/Wooden box 1.flac',
 '../Databases/IR/MeldaProduction_Box/Radio 02.flac',
 '../Databases/IR/MeldaProduction_Box/Bravo.flac',
 '../Databases/IR/MeldaProduction_Box/Guitar body.flac',
 '../Databases/IR/MeldaProduction_Box/Mobile Phone 03.flac',
 '../Databases/IR/MeldaProduction_Box/Gamma.flac',
 '../Databases/IR/MeldaProduction_Box/Classic.flac',
 '../Databases/IR/MeldaProduction_Box/Radio 01.flac',
 '../Databases/IR/MeldaProduction_Box/G12M70.flac',
 '../Databases/IR/MeldaProduction_Box/Legend 121.flac',
 '../Databases/IR/MeldaProduction_Box/G12M70 (hole).flac',
 '../Databases/IR/MeldaProd

# APPLY REVERBERATIONS

In [54]:
from scipy.signal import fftconvolve

In [104]:
raw_audios


['../Databases/Raw_database/Lily_Telecaster_108_DI.wav',
 '../Databases/Raw_database/Century_Larrivee_130_DI.wav',
 '../Databases/Raw_database/Lily_Larrivee_108_DI.wav',
 '../Databases/Raw_database/Mountain_Eastman_104_DI.wav',
 '../Databases/Raw_database/Lily_Ibanez_108_DI.wav',
 '../Databases/Raw_database/Century_Telecaster_130_DI.wav',
 '../Databases/Raw_database/Where_Larrivee_100_DI.wav',
 '../Databases/Raw_database/Hole_Larrivee_102_DI.wav',
 '../Databases/Raw_database/Train_Epiphone_112_DI.wav',
 '../Databases/Raw_database/Train_Eastman_112_DI.wav',
 '../Databases/Raw_database/Train_Ibanez_112_DI.wav',
 '../Databases/Raw_database/Hole_Eastman_102_DI.wav',
 '../Databases/Raw_database/Century_Eastman_130_DI.wav',
 '../Databases/Raw_database/Lily_Epiphone_108_DI.wav',
 '../Databases/Raw_database/Mountain_Telecaster_104_DI.wav',
 '../Databases/Raw_database/Where_Eastman_100_DI.wav',
 '../Databases/Raw_database/Mountain_Ibanez_104_DI.wav',
 '../Databases/Raw_database/Where_Telecaster

In [65]:
store_path = '/media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/MeldaProduction_Box/'
 
for aud in raw_audios:
    print('Processing {}'.format(aud))
    audio = ess.MonoLoader(filename=aud, sampleRate=sample_rate)()
    len_ref = len(audio)

    for ir,rev in zip(ir_files,ir_names):
        print('-> {}...'.format(rev))
        ir_samples = ess.MonoLoader(filename=ir, sampleRate=sample_rate)()

        output_file_name = store_path + aud.split('/')[-1].replace('.wav','_') +rev + '.wav'

    
#         audio_augmented = np.convolve (audio, ir_samples)
        audio_augmented = fftconvolve(audio, ir_samples, mode = 'full')    
        ess.MonoWriter(filename=output_file_name, format='wav')(audio_augmented)
       

Processing ../Databases/Raw_database/Lily_Telecaster_108_DI.wav
-> Mobile_Phone_01...
-> Guitar_Cabinet_03...
-> Guitar_Cabinet_02...
-> Voice_Recorder_01...
-> Guitar_Cabinet_01...
-> Laptop_Speaker...
-> Wooden_box_1...
-> Radio_02...
-> Bravo...
-> Guitar_body...
-> Mobile_Phone_03...
-> Gamma...
-> Classic...
-> Radio_01...
-> G12M70...
-> Legend_121...
-> G12M70_(hole)...
-> Wooden_box_2...
Processing ../Databases/Raw_database/Century_Larrivee_130_DI.wav
-> Mobile_Phone_01...
-> Guitar_Cabinet_03...
-> Guitar_Cabinet_02...
-> Voice_Recorder_01...
-> Guitar_Cabinet_01...
-> Laptop_Speaker...
-> Wooden_box_1...
-> Radio_02...
-> Bravo...
-> Guitar_body...
-> Mobile_Phone_03...
-> Gamma...
-> Classic...
-> Radio_01...
-> G12M70...
-> Legend_121...
-> G12M70_(hole)...
-> Wooden_box_2...
Processing ../Databases/Raw_database/Lily_Larrivee_108_DI.wav
-> Mobile_Phone_01...
-> Guitar_Cabinet_03...
-> Guitar_Cabinet_02...
-> Voice_Recorder_01...
-> Guitar_Cabinet_01...
-> Laptop_Speaker...


-> Bravo...
-> Guitar_body...
-> Mobile_Phone_03...
-> Gamma...
-> Classic...
-> Radio_01...
-> G12M70...
-> Legend_121...
-> G12M70_(hole)...
-> Wooden_box_2...
Processing ../Databases/Raw_database/Lily_Eastman_108_DI.wav
-> Mobile_Phone_01...
-> Guitar_Cabinet_03...
-> Guitar_Cabinet_02...
-> Voice_Recorder_01...
-> Guitar_Cabinet_01...
-> Laptop_Speaker...
-> Wooden_box_1...
-> Radio_02...
-> Bravo...
-> Guitar_body...
-> Mobile_Phone_03...
-> Gamma...
-> Classic...
-> Radio_01...
-> G12M70...
-> Legend_121...
-> G12M70_(hole)...
-> Wooden_box_2...
Processing ../Databases/Raw_database/Hole_Epiphone_102_DI.wav
-> Mobile_Phone_01...
-> Guitar_Cabinet_03...
-> Guitar_Cabinet_02...
-> Voice_Recorder_01...
-> Guitar_Cabinet_01...
-> Laptop_Speaker...
-> Wooden_box_1...
-> Radio_02...
-> Bravo...
-> Guitar_body...
-> Mobile_Phone_03...
-> Gamma...
-> Classic...
-> Radio_01...
-> G12M70...
-> Legend_121...
-> G12M70_(hole)...
-> Wooden_box_2...
Processing ../Databases/Raw_database/Hole_Tel

# APPLY EFFECTS

Chorus

In [102]:
store_path = '../Databases/Augmented_database/SOX_Effects/'
for aud in raw_audios:
# # Chorus
    tfm = sox.Transformer()
    tfm.set_globals(guard = True)
    tfm.chorus()
    output_file_name = store_path+ aud.split('/')[-1].replace('.wav','_').replace('_DI_','_')
    tfm.build_file(aud, output_file_name +'Chorus'+ '.wav')

# Flanger

    tfm = sox.Transformer()
    tfm.set_globals(guard = True)
    tfm.flanger(delay= 30, depth= 10, regen= 0, width= 80, speed= 0.4, phase = 50)

    tfm.build_file(aud,output_file_name +'Flanger'+ '.wav')

# # Overdrive

    tfm = sox.Transformer()
    tfm.set_globals(guard = False)
    tfm.overdrive(gain_db = 50, colour = 20)

    tfm.build_file(aud,output_file_name +'OverDrive'+ '.wav')

# # Phaser

    tfm = sox.Transformer()
    tfm.set_globals(guard = True)
    tfm.phaser(gain_in  = 0.8, gain_out = 1,delay = 5,speed = 1.2,modulation_shape = 'sinusoidal')

    tfm.build_file(aud,output_file_name +'Phaser'+ '.wav')

output_file: /media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/Effects/Lily_Telecaster_108_Flanger.wav already exists and will be overwritten on build
output_file: /media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/Effects/Lily_Telecaster_108_OverDrive.wav already exists and will be overwritten on build
output_file: /media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/Effects/Lily_Telecaster_108_Phaser.wav already exists and will be overwritten on build
output_file: /media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/Effects/Century_Larrivee_130_Flanger.wav already exists and will be overwritten on build
output_file: /media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/Effects/Century_Larrivee_130_OverDrive.wav already exists and will be overwritten on build
output_file: /media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/Effects/Century_Larrivee_130_Phaser.wav already exists and will be overwritten on build
output_file: /media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/Effects/Lily_Larrivee_108_Flanger.wav already e

output_file: /media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/Effects/Hole_Ibanez_102_Phaser.wav already exists and will be overwritten on build
output_file: /media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/Effects/Mountain_Epiphone_104_Flanger.wav already exists and will be overwritten on build
output_file: /media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/Effects/Mountain_Epiphone_104_OverDrive.wav already exists and will be overwritten on build
output_file: /media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/Effects/Mountain_Epiphone_104_Phaser.wav already exists and will be overwritten on build
output_file: /media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/Effects/Century_Epiphone_130_Flanger.wav already exists and will be overwritten on build
output_file: /media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/Effects/Century_Epiphone_130_OverDrive.wav already exists and will be overwritten on build
output_file: /media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/Effects/Century_Epiphone_130_Phaser.wav alread


# APPLY FILTERS

In [10]:
store_path = '../Databases/Augmented_database/SOX_Filters/'
freq_h = [500, 1000, 3000]
freq_l = [500,250,100]
boost = [-20,-10,10,20]
for audio_path in raw_audios:
    output_file = store_path+ audio_path.split('/')[-1].replace('.wav','_').replace('_DI_','_')
# High pass
    for f in freq_h:
        tfm = sox.Transformer()
        tfm.set_globals(guard = True)
        tfm.highpass(frequency = f)
        
        
        tfm.build_file(audio_path, output_file+'Hpass'+str(f)+ '.wav')


# Low pass
    for f in freq_l:
        tfm = sox.Transformer()
        tfm.set_globals(guard = True)
        tfm.lowpass(frequency = f)

        tfm.build_file(audio_path, output_file +'Lpass'+str(f)+ '.wav')


# # Treble
#     for b in boost:
#         tfm = sox.Transformer()
#         tfm.set_globals(guard = True)
#         tfm.treble(gain_db = b)

#         tfm.build_file(audio_path,output_file +'Treble'+str(b)+ '.wav')
# # Bass
#     for b in boost:
#         tfm = sox.Transformer()
#         tfm.set_globals(guard = True)
#         tfm.bass(gain_db = b)

#         tfm.build_file(audio_path,output_file +'Bass'+str(b)+ '.wav')
    
    


# APPLY GAUSSIAN RANDOM NOISE

In [68]:
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
import numpy as np

NOISES = 5
SAMPLE_RATE = 441000

for n in range(0,NOISES):
    augment = Compose([
        AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.1),])

    # Augment/transform/perturb the audio data
    augmented_samples = augment(samples=audio, sample_rate=SAMPLE_RATE)

    ess.MonoWriter(filename= './Reverbs/'+ audio_path.split('/')[-1].replace('.wav','_')+'noise'+str(n)+'.wav', format='wav')(augmented_samples)